In [ ]:
from nilearn import datasets
import pandas as pd

data = datasets.fetch_coords_seitzman_2018(legacy_format=False)

r = data['rois'][(data['networks'] == 'FrontoParietal') & (data['regions'] != 'cortexL') & (data['regions'] != 'cortexR')].copy()
# r["regions"] = data['regions'][(data['networks'] == 'FrontoParietal') & (data['regions'] != 'cortexL') & (data['regions'] != 'cortexR')]
# r["coords"] = r.apply(lambda x: (x['x'], x['y'], x['z']), axis=1)
# dmn_coords = r['coords'].tolist()

data

In [ ]:
# CinguloOpercular
# SomatomotorDorsal
# FrontoParietal

dmn_coords = [[
 (32, -81, -38),
 (-32, -78, -38),
 (-24, -76, -28),
 (24, -76, -28),
 (-6, -51, -41),
 (8, -50, -40),
 (-25, -39, -2),
 (25, -37, -2),
 (-5, -10, 9),
 (4, -8, 8),
 (-13, 17, 7),
 (12, 18, 7)],
 [
 (44, -60, -30),
 (-44, -60, -30),
 (-34, -42, -44),
 (-15, -14, 12),
 (13, -14, 12),
 (-9, -10, 0),
 (10, -8, 2)
 ],
[(-6, -74, -42),
 (8, -72, -39),
 (-10, -62, -18),
 (10, -62, -18),
 (-33, -51, -50),
 (-12, -44, -18),
 (12, -44, -18),
 (-19, -23, 10),
 (16, -22, 9),
 (-14, -20, 1),
 (14, -19, 0),
 (-28, -10, -4),
 (-28, -10, 9),
 (29, -8, 8),
 (28, -7, -5),
 (-19, -5, -3),
 (19, -5, -4),
 (-28, -1, -3),
 (25, 2, -1),
 (25, 5, 7),
 (-25, 8, 8)],
 [(-10, -78, -28),
 (10, -78, -28),
 (-34, -72, -48),
 (34, -72, -48),
 (-31, -66, -30),
 (32, -63, -30),
 (40, -44, -38),
 (-15, -2, 19),
 (14, -1, 18)]
]


In [ ]:
brain_map_network = dict()
for i in range(12):
    brain_map_network[i] = 'DMN'
for i in range(7):
    brain_map_network[12+i] = 'CON'
for i in range(21):
    brain_map_network[19+i] = 'SDN'
for i in range(9):
    brain_map_network[40+i] = 'FPN'
brain_map_network

Healthy Control - Load data

In [ ]:
import nibabel as nib
from nilearn.maskers import NiftiSpheresMasker
from nilearn.maskers.nifti_spheres_masker import _iter_signals_from_spheres
from scipy import stats
import numpy as np
import scipy
import matplotlib.pyplot as plt

subjects = [10159, 10171, 10189, 10206, 10217, 10225, 10227, 10228, 10235, 10249, 10269,
            10271, 10273, 10274, 10280, 10290, 10292, 10299, 10304, 10316, 10321, 10325, 10329, 10339, 10340,
            10345, 10347, 10356, 10361, 10365, 10376, 10428, 10429, 10438, 10440, 10448, 10455, 10460, 10471, 10478,
            10492, 10501, 10506, 10517, 10523, 10525, 10527, 10530]

# All networks
all_coords = [c for coords in dmn_coords for c in coords]

timeseries_all = {}

for subj in subjects:
    name = "warped_img_sub-" + str(subj) + "_all.nii"
    nifti = nib.load(name)

    print("Processing ... ", str(subj))

    # Sphere of a radius
    masker = NiftiSpheresMasker(all_coords, radius=4, standardize=True, memory="nilearn_cache", verbose=2)
    masker.fit()
    sphere_timeseries = _iter_signals_from_spheres(
        masker.seeds_, 
        nifti,
        masker.radius,
        masker.allow_overlap)

    timeseries_all[subj] = list(sphere_timeseries)

In [ ]:
# from joblib import dump, load
# import pandas as pd

# pickle_file = './all_beta_mean_timeseries_all_rois.joblib'
# with open(pickle_file, 'wb') as f:
#     dump(timeseries_all, f, compress='zlib')

In [ ]:
# from joblib import dump, load

# pickle_file = './all_beta_mean_timeseries_all_rois.joblib'
# with open(pickle_file, 'rb') as f:
#     timeseries_all = load(f)

In [ ]:
len(timeseries_all)

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

subject = 10159
# subject = 10345
# subject = 10304
# subject = 10273
roi = 8
# roi = 12
# roi = 20

fig, axs = plt.subplots(1, 3, constrained_layout = True)
roi8 = pd.DataFrame(timeseries_all[subject][roi]).iloc[10]
roi8.hist(grid=True, bins=20, rwidth=0.9, alpha=0.5, ax=axs[0], density=True)

params = scipy.stats.norm.fit(roi8)
x = sorted(roi8)
norm_pdf = stats.norm.pdf(x, params[0], params[1])
print(params[0], params[1])
axs[0].plot(x, norm_pdf, 'r')

roi8 = pd.DataFrame(timeseries_all[subject][roi]).iloc[50]
roi8.hist(grid=True, bins=20, rwidth=0.9, alpha=0.5, ax=axs[1], density=True)

params = scipy.stats.norm.fit(roi8)
x = sorted(roi8)
norm_pdf = stats.norm.pdf(x, params[0], params[1])
print(params[0], params[1])
axs[1].plot(x, norm_pdf, 'r')

roi8 = pd.DataFrame(timeseries_all[subject][roi]).iloc[100]
roi8.hist(grid=True, bins=20, rwidth=0.9, alpha=0.5, ax=axs[2], density=True)

params = scipy.stats.norm.fit(roi8)
x = sorted(roi8)
norm_pdf = stats.norm.pdf(x, params[0], params[1])
print(params[0], params[1])
axs[2].plot(x, norm_pdf, 'r')

axs[1].set_xlabel("rs-fMRI Signal")
axs[0].set_ylabel("Frequency")
axs[0].set_title("Time Point = 10")
axs[1].set_title("Time Point = 50")
axs[2].set_title("Time Point = 100")


# QQ-plot
import statsmodels.api as sm

fig, axs = plt.subplots(1, 3, constrained_layout = True)
roi8 = pd.DataFrame(timeseries_all[subject][roi]).iloc[10]
fig = sm.qqplot((roi8-roi8.mean())/roi8.std(), line='45', ax=axs[0])
roi8 = pd.DataFrame(timeseries_all[subject][roi]).iloc[50]
fig = sm.qqplot((roi8-roi8.mean())/roi8.std(), line='45', ax=axs[1])
roi8 = pd.DataFrame(timeseries_all[subject][roi]).iloc[100]
fig = sm.qqplot((roi8-roi8.mean())/roi8.std(), line='45', ax=axs[2])
axs[0].set_title("Time Point = 10")
axs[1].set_title("Time Point = 50")
axs[2].set_title("Time Point = 100")
plt.show()

SZ - Load data

In [ ]:
import nibabel as nib
from nilearn.maskers import NiftiSpheresMasker
from nilearn.maskers.nifti_spheres_masker import _iter_signals_from_spheres
from scipy import stats
import numpy as np
import scipy
import matplotlib.pyplot as plt

szsubjects = [50004, 50005, 50006, 50007, 50008, 50010, 50013, 50014, 50015, 50016, 50020,
            50021, 50022, 50023, 50025, 50027, 50029, 50032, 50033, 50034, 50035, 50036,
            50038, 50043, 50047, 50048, 50049, 50050, 50051, 50052, 50053, 50054, 50055, 50056, 50059, 50060, 50061,
            50064, 50066, 50067, 50069, 50073, 50075, 50076, 50077, 50080, 50081, 50083, 50085]

sztimeseries_all = {}

# All networks
all_coords = [c for coords in dmn_coords for c in coords]

for subj in szsubjects:
    name = "warped_img_sub-" + str(subj) + "_all.nii"
    nifti = nib.load(name)

    print("Processing ... ", str(subj))

    # Sphere of a radius
    masker = NiftiSpheresMasker(all_coords, radius=4, standardize=True, memory="nilearn_cache", verbose=2)
    masker.fit()
    sphere_timeseries = _iter_signals_from_spheres(
        masker.seeds_, 
        nifti,
        masker.radius,
        masker.allow_overlap)

    sztimeseries_all[subj] = list(sphere_timeseries)

In [ ]:
# from joblib import dump, load
# import pandas as pd

# pickle_file = './all_beta_mean_sztimeseries_all_rois.joblib'
# with open(pickle_file, 'wb') as f:
#     dump(sztimeseries_all, f, compress='zlib')

In [ ]:
# from joblib import dump, load

# pickle_file = './all_beta_mean_sztimeseries_all_rois.joblib'
# with open(pickle_file, 'rb') as f:
#     sztimeseries_all = load(f)

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

subject = 50007
# subject = 10345
# subject = 10304
# subject = 10273
roi = 8
# roi = 12
# roi = 20
# roi = 39

fig, axs = plt.subplots(1, 3, constrained_layout = True)
roi8 = pd.DataFrame(sztimeseries_all[subject][roi]).iloc[10]
roi8.hist(grid=True, bins=20, rwidth=0.9, alpha=0.5, ax=axs[0], density=True)

params = scipy.stats.norm.fit(roi8)
x = sorted(roi8)
norm_pdf = stats.norm.pdf(x, params[0], params[1])
print(params[0], params[1])
axs[0].plot(x, norm_pdf, 'r')

roi8 = pd.DataFrame(sztimeseries_all[subject][roi]).iloc[50]
roi8.hist(grid=True, bins=20, rwidth=0.9, alpha=0.5, ax=axs[1], density=True)

params = scipy.stats.norm.fit(roi8)
x = sorted(roi8)
norm_pdf = stats.norm.pdf(x, params[0], params[1])
print(params[0], params[1])
axs[1].plot(x, norm_pdf, 'r')

roi8 = pd.DataFrame(sztimeseries_all[subject][roi]).iloc[100]
roi8.hist(grid=True, bins=20, rwidth=0.9, alpha=0.5, ax=axs[2], density=True)

params = scipy.stats.norm.fit(roi8)
x = sorted(roi8)
norm_pdf = stats.norm.pdf(x, params[0], params[1])
print(params[0], params[1])
axs[2].plot(x, norm_pdf, 'r')

axs[1].set_xlabel("rs-fMRI Signal")
axs[0].set_ylabel("Frequency")
axs[0].set_title("Time Point = 10")
axs[1].set_title("Time Point = 50")
axs[2].set_title("Time Point = 100")


# QQ-plot
import statsmodels.api as sm

fig, axs = plt.subplots(1, 3, constrained_layout = True)
roi8 = pd.DataFrame(timeseries_all[subject][roi]).iloc[10]
fig = sm.qqplot((roi8-roi8.mean())/roi8.std(), line='45', ax=axs[0])
roi8 = pd.DataFrame(timeseries_all[subject][roi]).iloc[50]
fig = sm.qqplot((roi8-roi8.mean())/roi8.std(), line='45', ax=axs[1])
roi8 = pd.DataFrame(timeseries_all[subject][roi]).iloc[100]
fig = sm.qqplot((roi8-roi8.mean())/roi8.std(), line='45', ax=axs[2])
axs[0].set_title("Time Point = 10")
axs[1].set_title("Time Point = 50")
axs[2].set_title("Time Point = 100")
plt.show()

Searching

In [ ]:
import itertools 

all_region_pairs = list(itertools.combinations(range(len(all_coords)), 2))

In [ ]:
all_corrs_np.shape

In [ ]:
import nibabel as nib
from nilearn.maskers import NiftiSpheresMasker
from nilearn.maskers.nifti_spheres_masker import _iter_signals_from_spheres
from scipy import stats
import numpy as np
import scipy
import matplotlib.pyplot as plt
from tqdm import tqdm
import pandas as pd
from scipy.stats import beta 
from scipy.optimize import curve_fit
from itertools import product

# Minimum number of voxels
# all_v = []
# for x in timeseries_all[subjects[0]]:
#     all_v.append(len(x[0]))
# min_v = min(all_v)
# min_v

all_beta_mean = dict()
all_beta_variance = dict()

# All pairs
for regions in tqdm(all_region_pairs): # tqdm([(8,9)]):

    # beta mean
    beta_mean = np.array([])
    beta_variance = np.array([])

    print("HC Processing ... region:", regions)
    
    for subj in [10159]: #subjects: # [10159]:

        # Correlation Analysis
        roi_1, roi_2 = regions

        ts1 = timeseries_all[subj][roi_1]
        ts2 = timeseries_all[subj][roi_2]
        
        df1 = pd.DataFrame(ts1)
        df2 = pd.DataFrame(ts2)

        # time
        T = ts1.shape[0]
        # voxels
        V = ts1.shape[1] # min_v
        # sliding window
        window_size = 35

        # correlation
        all_corrs = []

        # for all the voxels (all pairs)
#         for v in range(V):

        cartesian_product = product(range(V), repeat=2)
        for a, b in cartesian_product:
            # correlation for the same voxel between 2 regions
            corrs = df1[a].rolling(window_size).corr(df2[b])
            corrs = corrs.to_frame()
            all_corrs.append(corrs[window_size-1:].iloc[:,0].tolist())

        # convert it to np array
        all_corrs_np = np.array(all_corrs).transpose()

        # Beta parameters
        beta_all = []

        for t in range(all_corrs_np.shape[0]):
            beta_params = scipy.stats.beta.fit(all_corrs_np[t,:], floc=-1, fscale=2)
            beta_all.append(beta_params)
            
            # standard errors for paramters
#             fig, ax = plt.subplots() 
#             nbins = 100
#             n, bins, patches = ax.hist(all_corrs_np[t,:], nbins, density=True, facecolor = 'grey', alpha = 0.5, label='before'); 
#             centers = (0.5*(bins[1:]+bins[:-1]))
#             pars, cov = curve_fit(lambda x, a, b : beta.pdf(x, a, b, -1, 2), centers, n, p0=[beta_params[0],beta_params[1]])
#             beta_all.append([pars[0], pars[1], 2*np.sqrt(cov[0,0]), 2*np.sqrt(cov[1,1])])

        beta_all_np = np.array(beta_all)

        beta_mean = np.append(beta_mean, (beta_all_np[:, 0] - beta_all_np[:, 1]) / (beta_all_np[:, 0] + beta_all_np[:, 1]))
        beta_variance = np.append(beta_variance, 
                                  4 * beta_all_np[:, 0] * beta_all_np[:, 1] / ((beta_all_np[:, 0] + beta_all_np[:, 1])**2 * (beta_all_np[:, 0] + beta_all_np[:, 1] + 1)))
    
    beta_roi = str(roi_1) + "," + str(roi_2)
    all_beta_mean[beta_roi] = beta_mean
    all_beta_variance[beta_roi] = beta_variance
    
sz_all_beta_mean = dict()

# All pairs
for regions in tqdm(all_region_pairs):

    # beta mean
    beta_mean = np.array([])

    print("SZ Processing ... region:", regions)
    
    for subj in szsubjects:

        # Correlation Analysis
        roi_1, roi_2 = regions

        ts1 = sztimeseries_all[subj][roi_1]
        ts2 = sztimeseries_all[subj][roi_2]
        
        df1 = pd.DataFrame(ts1)
        df2 = pd.DataFrame(ts2)

        # time
        T = ts1.shape[0]
        # voxels
        V = ts1.shape[1]
        # sliding window
        window_size = 35

        # correlation
        all_corrs = []

        # for all the voxels
#         for v in range(V):
#             # correlation for the same voxel between 2 regions
#             corrs = df1[[v]].rolling(window_size).corr(df2[[v]])
#             all_corrs.append(corrs[window_size-1:].iloc[:,0].tolist())

        cartesian_product = product(range(V), repeat=2)
        for a, b in cartesian_product:
            # correlation for the same voxel between 2 regions
            corrs = df1[a].rolling(window_size).corr(df2[b])
            corrs = corrs.to_frame()
            all_corrs.append(corrs[window_size-1:].iloc[:,0].tolist())
            
        # convert it to np array
        all_corrs_np = np.array(all_corrs).transpose()
        
        # Beta parameters
        beta_all = []

        for t in range(all_corrs_np.shape[0]):
            beta_params = scipy.stats.beta.fit(all_corrs_np[t,:], floc=-1, fscale=2)
            beta_all.append(beta_params)

        beta_all_np = np.array(beta_all)

        beta_mean = np.append(beta_mean, (beta_all_np[:, 0] - beta_all_np[:, 1]) / (beta_all_np[:, 0] + beta_all_np[:, 1]))
    
    beta_roi = str(roi_1) + "," + str(roi_2)
    sz_all_beta_mean[beta_roi] = beta_mean

In [ ]:
# np.var(all_corrs_np, axis=1)

# all_beta_variance

In [ ]:
# df = pd.DataFrame({"Beta variance": all_beta_variance['8,9'], "Sample variance": np.var(all_corrs_np, axis=1)})
# df.plot(xlabel="Time Point", ylabel="Variance", title="Sliding-window Variance, HC subject ID 10159, between Thalamus ROI 8 and ROI 9")
# plt.grid()

In [ ]:
df = pd.DataFrame(all_corrs_np35)
df.plot(legend=False, xlabel="Time Point", ylabel="Correlation Coefficient")
plt.grid()

In [ ]:
import scipy

fig, axs = plt.subplots(1, 3, constrained_layout = True)

s = pd.Series(all_corrs_np[10,:])
s.hist(grid=True, bins=20, rwidth=0.9, alpha=0.5, density=True, ax=axs[0])

beta_params = scipy.stats.beta.fit(s, floc=-1, fscale=2)
x = sorted(s)
beta_pdf = stats.beta.pdf(x, beta_params[0], beta_params[1], beta_params[2], beta_params[3])
print(beta_params[0], beta_params[1], beta_params[2], beta_params[3])
axs[0].plot(x, beta_pdf, 'r')

####################################################

s = pd.Series(all_corrs_np[50,:])
s.hist(grid=True, bins=20, rwidth=0.9, alpha=0.5, density=True, ax=axs[1])

beta_params = scipy.stats.beta.fit(s, floc=-1, fscale=2)
x = sorted(s)
beta_pdf = stats.beta.pdf(x, beta_params[0], beta_params[1], beta_params[2], beta_params[3])
print(beta_params[0], beta_params[1], beta_params[2], beta_params[3])
axs[1].plot(x, beta_pdf, 'r')

####################################################

s = pd.Series(all_corrs_np[100,:])
s.hist(grid=True, bins=20, rwidth=0.9, alpha=0.5, density=True, ax=axs[2])

beta_params = scipy.stats.beta.fit(s, floc=-1, fscale=2)
x = sorted(s)
beta_pdf = stats.beta.pdf(x, beta_params[0], beta_params[1], beta_params[2], beta_params[3])
print(beta_params[0], beta_params[1], beta_params[2], beta_params[3])
axs[2].plot(x, beta_pdf, 'r')

axs[1].set_xlabel("Correlation Coefficient")
axs[0].set_ylabel("Probability Density")
axs[0].set_title("Time Point = 10")
axs[1].set_title("Time Point = 50")
axs[2].set_title("Time Point = 100")


In [ ]:
import scipy

fig, axs = plt.subplots(1, 3, constrained_layout = True)

s = pd.Series(all_corrs_np[10,:])
s.hist(grid=True, bins=20, rwidth=0.9, alpha=0.5, density=True, ax=axs[0])

beta_params = scipy.stats.beta.fit(s, floc=-1, fscale=2)
x = sorted(s)
beta_pdf = stats.beta.pdf(x, beta_params[0], beta_params[1], beta_params[2], beta_params[3])
print(beta_params[0], beta_params[1], beta_params[2], beta_params[3])
axs[0].plot(x, beta_pdf, 'r')

####################################################

s = pd.Series(all_corrs_np[50,:])
s.hist(grid=True, bins=20, rwidth=0.9, alpha=0.5, density=True, ax=axs[1])

beta_params = scipy.stats.beta.fit(s, floc=-1, fscale=2)
x = sorted(s)
beta_pdf = stats.beta.pdf(x, beta_params[0], beta_params[1], beta_params[2], beta_params[3])
print(beta_params[0], beta_params[1], beta_params[2], beta_params[3])
axs[1].plot(x, beta_pdf, 'r')

####################################################

s = pd.Series(all_corrs_np[100,:])
s.hist(grid=True, bins=20, rwidth=0.9, alpha=0.5, density=True, ax=axs[2])

beta_params = scipy.stats.beta.fit(s, floc=-1, fscale=2)
x = sorted(s)
beta_pdf = stats.beta.pdf(x, beta_params[0], beta_params[1], beta_params[2], beta_params[3])
print(beta_params[0], beta_params[1], beta_params[2], beta_params[3])
axs[2].plot(x, beta_pdf, 'r')

axs[1].set_xlabel("Correlation Coefficient")
axs[0].set_ylabel("Probability Density")
axs[0].set_title("Time Point = 10")
axs[1].set_title("Time Point = 50")
axs[2].set_title("Time Point = 100")


In [ ]:
#######################
#######################
#######################

import scipy

fig, axs = plt.subplots(1, 3, constrained_layout = True)

s = pd.Series(all_corrs_np[10,:])
s.hist(grid=True, bins=20, rwidth=0.9, alpha=0.5, density=True, ax=axs[0])

beta_params = scipy.stats.beta.fit(s, floc=-1, fscale=2)
x = sorted(s)
beta_pdf = stats.beta.pdf(x, beta_params[0], beta_params[1], beta_params[2], beta_params[3])
print(beta_params[0], beta_params[1], beta_params[2], beta_params[3])
axs[0].plot(x, beta_pdf, 'r')

####################################################

s = pd.Series(all_corrs_np[50,:])
s.hist(grid=True, bins=20, rwidth=0.9, alpha=0.5, density=True, ax=axs[1])

beta_params = scipy.stats.beta.fit(s, floc=-1, fscale=2)
x = sorted(s)
beta_pdf = stats.beta.pdf(x, beta_params[0], beta_params[1], beta_params[2], beta_params[3])
print(beta_params[0], beta_params[1], beta_params[2], beta_params[3])
axs[1].plot(x, beta_pdf, 'r')

####################################################

s = pd.Series(all_corrs_np[100,:])
s.hist(grid=True, bins=20, rwidth=0.9, alpha=0.5, density=True, ax=axs[2])

beta_params = scipy.stats.beta.fit(s, floc=-1, fscale=2)
x = sorted(s)
beta_pdf = stats.beta.pdf(x, beta_params[0], beta_params[1], beta_params[2], beta_params[3])
print(beta_params[0], beta_params[1], beta_params[2], beta_params[3])
axs[2].plot(x, beta_pdf, 'r')

axs[1].set_xlabel("Correlation Coefficient")
axs[0].set_ylabel("Probability Density")
axs[0].set_title("Time Point = 10")
axs[1].set_title("Time Point = 50")
axs[2].set_title("Time Point = 100")


In [ ]:
import scipy
import math

fig, axs = plt.subplots(1, 3, constrained_layout = True)

# two-sample Kolmogorov-Smirnov test result
test_results = []

# for all timepoints
for i in range(len(all_corrs_np)):
    test_results.append(scipy.stats.ks_2samp(all_corrs_np[10,:], all_corrs_np[i,:]).pvalue)

test_results_005_level = [x < 0.05/118 for x in test_results]
colors = pd.Series(["r" if z else "b" for z in test_results_005_level])
test_results_log10 = [300 if z < 1e-300 else -math.log10(z) for z in test_results]
# colors = pd.Series(["r" if x < 1e-200 else "m" if x < 0.05 else "b" for x in test_results])

axs[0].scatter(range(len(all_corrs_np)), test_results_log10, c=colors)
axs[0].grid()

# print(test_results)

############################################################################

# two-sample Kolmogorov-Smirnov test result
test_results = []

# for all timepoints
for i in range(len(all_corrs_np)):
    test_results.append(scipy.stats.ks_2samp(all_corrs_np[50,:], all_corrs_np[i,:]).pvalue)

test_results_005_level = [x < 0.05/118 for x in test_results]
colors = pd.Series(["r" if z else "b" for z in test_results_005_level])
test_results_log10 = [300 if z < 1e-300 else -math.log10(z) for z in test_results]
# colors = pd.Series(["r" if x < 1e-200 else "m" if x < 0.05 else "b" for x in test_results])

axs[1].scatter(range(len(all_corrs_np)), test_results_log10, c=colors)
axs[1].grid()

################################################################################

# two-sample Kolmogorov-Smirnov test result
test_results = []

# for all timepoints
for i in range(len(all_corrs_np)):
    test_results.append(scipy.stats.ks_2samp(all_corrs_np[100,:], all_corrs_np[i,:]).pvalue)

test_results_005_level = [x < 0.05/118 for x in test_results]
colors = pd.Series(["r" if z else "b" for z in test_results_005_level])
test_results_log10 = [300 if z < 1e-300 else -math.log10(z) for z in test_results]
# colors = pd.Series(["r" if x < 1e-200 else "m" if x < 0.05 else "b" for x in test_results])

axs[2].scatter(range(len(all_corrs_np)), test_results_log10, c=colors)
axs[2].grid()

axs[1].set_xlabel("Time Point")
axs[0].set_ylabel(r"$-\log_{10}(\mathrm{p\text{-}value})$")
axs[0].set_title("Time Point = 10")
axs[1].set_title("Time Point = 50")
axs[2].set_title("Time Point = 100")

In [ ]:
len(all_corrs_np25)

In [ ]:
# from joblib import dump, load
# import pandas as pd

# df_beta = pd.DataFrame(data=all_beta_mean)

# pickle_file = './all_beta_mean_all_rois_window_35.joblib'
# with open(pickle_file, 'wb') as f:
#     dump(df_beta, f, compress='zlib')

In [ ]:
# from joblib import dump, load
# import pandas as pd

# df_beta_sz = pd.DataFrame(data=sz_all_beta_mean)

# pickle_file = './sz_all_beta_mean_all_rois_window_35.joblib'
# with open(pickle_file, 'wb') as f:
#     dump(df_beta_sz, f, compress='zlib')

In [ ]:
from joblib import dump, load

pickle_file = './all_beta_mean_all_rois_window_35.joblib'
with open(pickle_file, 'rb') as f:
    df_beta = load(f)

In [ ]:
from joblib import dump, load

pickle_file = './sz_all_beta_mean_all_rois_window_35.joblib'
with open(pickle_file, 'rb') as f:
    df_beta_sz = load(f)

In [ ]:
beta_mean = (beta_all_np[:, 0] - beta_all_np[:, 1]) / (beta_all_np[:, 0] + beta_all_np[:, 1])
x = np.arange(len(beta_mean))

beta_variance = (beta_all_np[:, 0]*beta_all_np[:, 1])*4 / ((beta_all_np[:, 0] + beta_all_np[:, 1])**2 * (beta_all_np[:, 0] + beta_all_np[:, 1] + 1))
beta_std_err = beta_variance ** 0.5 / (257*257)**0.5

plt.grid()
plt.errorbar(x, beta_mean, 2*beta_std_err, linestyle='None', marker='*')
plt.ylabel("Correlation Coefficient")
plt.xlabel("Time Point")

In [ ]:
(min(beta_mean), max(beta_mean), min(beta_std), max(beta_std))

In [ ]:
(min(beta_mean), max(beta_mean), min(beta_std_err), max(beta_std_err))

In [ ]:
beta_all_np

In [ ]:
df = pd.DataFrame({"α" : beta_all_np[:, 0], "β" : beta_all_np[:, 1]})
#df.plot()

# Change the way how beta_all is calculated earlier, using curve fit
x = np.arange(len(beta_all_np[:, 0]))
plt.grid()
plt.errorbar(x, beta_all_np[:, 0], beta_all_np[:, 2], linestyle='None', marker='*')
plt.errorbar(x, beta_all_np[:, 1], beta_all_np[:, 3], linestyle='None', marker='*')
plt.legend(["α", "β"])
plt.ylabel("Beta Paramter Values")
plt.xlabel("Time Point")

In [ ]:
from tqdm import tqdm
import scipy
import pandas as pd

# KS test
test_results = []

import scipy
for y in tqdm(list(df_beta)):
    c = df_beta[y]
    z = df_beta_sz[y]
    test_results.append(scipy.stats.ks_2samp(c, z).pvalue)

df_tests = pd.DataFrame({"ks test": test_results})
df_tests.index = list(df_beta)
print(df_tests)


In [ ]:
from tqdm import tqdm
import scipy
import pandas as pd

# KS test - Effect Size
test_results_d = []

import scipy
for y in tqdm(list(df_beta)):
    if -math.log10(df_tests["ks test"][y]) > 4.4:
        c = df_beta[y]
        z = df_beta_sz[y]
        n1 = c.shape[0]
        n2 = z.shape[0]
        test_results_d.append(scipy.stats.ks_2samp(c, z).statistic / (n1*n2/(n1+n2))**0.5)
    else:
        test_results_d.append(-1)

df_tests_d = pd.DataFrame({"ks test": test_results_d})
df_tests_d.index = list(df_beta)
print(df_tests_d)

In [ ]:
import matplotlib.pyplot as plt
import math
from collections import Counter

df_tests["ks test"].apply(lambda x : -math.log10(x)).plot.hist(grid=True, bins=20, rwidth=0.9, alpha=0.5)
plt.title("HC vs SZ")
plt.xlabel("-log10 p-value")
plt.ylabel("Count of ROI pairs")
plt.show()

bins = [0, 4.4]
bins.extend(list(range(25, 200, 25)))
x = df_tests["ks test"].apply(lambda x : -math.log10(x)).value_counts(bins=bins)
print(x)
print(x / x.sum() * 100)

# # KL divergence
# df_kld["kl divergence"].plot.hist(grid=True, bins=20, rwidth=0.9, alpha=0.5)
# plt.show()

# bins = list(range(0, 20, 3))
# x = df_kld["kl divergence"].value_counts(bins=bins)
# print(x / x.sum() * 100)

a = df_tests["ks test"].sort_values(ascending=True).index[0:50]
print("#################################################")
print("ks test:")
print(a)

network_pairs = []
for roi_pair in a:
    a1, a2 = roi_pair.split(',')
    network_pairs.append((brain_map_network[int(a1)], brain_map_network[int(a2)]))
print(Counter(network_pairs))

# print("#################################################")
# print("kl divergence:")
# b = df_kld["kl divergence"].sort_values(ascending=False).index[0:50]
# print(b)

# network_pairs = []
# for roi_pair in b:
#     a1, a2 = roi_pair.split(',')
#     network_pairs.append((brain_map_network[int(a1)], brain_map_network[int(a2)]))
# print(Counter(network_pairs))

# print("#################################################")
# print("Common to both:")
# both_tests_list = set(a).intersection(set(b))
# print(both_tests_list)

# network_pairs = []
# for roi_pair in both_tests_list:
#     a1, a2 = roi_pair.split(',')
#     network_pairs.append((brain_map_network[int(a1)], brain_map_network[int(a2)]))
# print(Counter(network_pairs))

In [ ]:
import matplotlib.pyplot as plt
import math
from collections import Counter

df_tests_d["ks test"].plot.hist(grid=True, bins=20, rwidth=0.9, alpha=0.5)
plt.title("HC vs SZ")
plt.xlabel("Effect size")
plt.ylabel("Count of ROI pairs")
plt.show()

bins = [-1, 0, 0.001, 0.002, 0.0035, 0.007, 1]
x = df_tests_d["ks test"].value_counts(bins=bins)
print(x)
print(x / x.sum() * 100)

# # KL divergence
# df_kld["kl divergence"].plot.hist(grid=True, bins=20, rwidth=0.9, alpha=0.5)
# plt.show()

# bins = list(range(0, 20, 3))
# x = df_kld["kl divergence"].value_counts(bins=bins)
# print(x / x.sum() * 100)

# a = df_tests_d["ks test"].sort_values(ascending=True).index[0:44]
a = df_tests_d["ks test"][df_tests_d["ks test"]>0.0035].index
print("#################################################")
print("ks test:")
print(a)

network_pairs = []
for roi_pair in a:
    a1, a2 = roi_pair.split(',')
    network_pairs.append((brain_map_network[int(a1)], brain_map_network[int(a2)]))
print(Counter(network_pairs))

# print("#################################################")
# print("kl divergence:")
# b = df_kld["kl divergence"].sort_values(ascending=False).index[0:50]
# print(b)

# network_pairs = []
# for roi_pair in b:
#     a1, a2 = roi_pair.split(',')
#     network_pairs.append((brain_map_network[int(a1)], brain_map_network[int(a2)]))
# print(Counter(network_pairs))

# print("#################################################")
# print("Common to both:")
# both_tests_list = set(a).intersection(set(b))
# print(both_tests_list)

# network_pairs = []
# for roi_pair in both_tests_list:
#     a1, a2 = roi_pair.split(',')
#     network_pairs.append((brain_map_network[int(a1)], brain_map_network[int(a2)]))
# print(Counter(network_pairs))

In [ ]:
df_tests["ks test"].shape

In [ ]:
import matplotlib.pyplot as plt
import scipy
import pandas as pd
import math
from collections import Counter

network_pairs = []

# for i, y in enumerate(df_tests["ks test"].sort_values(ascending=True).index[0:50]):
for i, y in enumerate(df_tests_d["ks test"][df_tests_d["ks test"]>0.0035].index):
    fig, ax = plt.subplots(1, 1)
    c = df_beta[y]
    z = df_beta_sz[y]
    
    df = pd.DataFrame({"HC": c, "BP": z})
    print("Mean: (HC) ", c.mean(), "(BP) ", z.mean())
    print("STD: (HC) ", c.std(), "(BP) ", z.std())
    print(-math.log10(scipy.stats.ks_2samp(c, z).pvalue))
    df.plot.hist(grid=True, bins=20, rwidth=0.9, ax=ax, alpha=0.5, density=True)
    ax.set_title("HC vs BP, Fitted means (Window = 35): ROI " + y + ", Top " + str(i+1) + ", -log10 p-value = " + str(round(-math.log10(scipy.stats.ks_2samp(c, z).pvalue))))
    ax.set_ylabel("Density")
    plt.show()
    
    a1, a2 = y.split(',')
    network_pairs.append((brain_map_network[int(a1)], brain_map_network[int(a2)], "hyper" if z.mean() > c.mean() else "hypo"))

print(Counter(network_pairs))

In [ ]:
from joblib import dump, load

pickle_file = './beta_testing_results_2.joblib'
with open(pickle_file, 'rb') as f:
    df_testing_results = load(f)